In [22]:
import math

class ShearDesign:
    '''
    ---- Design Beam Stirrups Spacing ------
    ShearDesign(beam_width(bw),
    effective_depth(d),
    compressive_strength(f'c),
    tensile_strength(fyt),
    type_of_concrete(lambda),
    stirrups_dia(ds),
    number_of_legs(n),
    Ultimate_Shear_capacity(Vu))
    '''
    def __init__(self, bw, d, fc, fyt, lamb, ds, n, Vu):
        self.bw = bw
        self.d = d
        self.fc = fc
        self.fyt = fyt
        self.lamb = self.calculate_lambda(lamb)
        self.Av = self.stirrups_area(ds, n)
        self.Vu = Vu  # Must be in Newtons
        self.Vc = self.concrete_shear(self.lamb, fc, bw, d)  # Must be in Newtons
        self.Vs = self.stirrups_shear(Vu, self.Vc)  # Must be in Newtons
         
    def calculate_lambda(self, lambda_constant):
        if lambda_constant == 'Normal Weight Concrete':
            return 1
        elif lambda_constant == 'Light Weight Concrete':
            return 0.75
        elif lambda_constant == 'Sand Light Weight Concrete':
            return 0.85
        else:
            raise ValueError('Invalid Input!')
            
    def stirrups_area(self, diameter, n_legs):
        return n_legs * (math.pi / 4) * diameter**2
        
    def concrete_shear(self, lambda_constant, fc, bw, d):
        return (1 / 6) * lambda_constant * math.sqrt(fc) * bw * d

    def stirrups_shear(self, Vu, Vc):
        return (Vu / 0.75) - Vc

    def stirrups_spacing(self):
        '''
        --No Input--
        Calculate maximum spacing & rounding off to the nearest 25 mm.
        '''
        if (0.75 / 2) * self.Vc >= self.Vu:
            return 'Stirrups not needed.'
        elif (0.75 / 2) * self.Vc < self.Vu and self.Vu <= 0.75 * self.Vc:
            smax = min((self.Av * self.fyt) / (0.062 * math.sqrt(self.fc) * self.bw), 
                       (self.Av * self.fyt) / (0.35 * self.bw), self.d / 2, 600)
        elif 0.75 * self.Vc < self.Vu and self.Vu <= 3 * 0.75 * self.Vc:
            smax = min((self.Av * self.fyt) / (0.062 * math.sqrt(self.fc) * self.bw), 
                       (self.Av * self.fyt) / (0.35 * self.bw), self.d / 2, 600, 
                       (self.Av * self.fyt * self.d) / self.Vs)
        elif 3 * 0.75 * self.Vc < self.Vu and self.Vu <= 5 * 0.75 * self.Vc:
            smax = min((self.Av * self.fyt * self.d) / self.Vs, self.d / 4, 600)
        elif self.Vu > 5 * 0.75 * self.Vc:
            return 'Revise Beam Dimension!'

        smax = math.floor(smax / 25) * 25
        return smax

#---------------------------------------Inputs----------------------------------------------

# Define the input parameters
bw = 350  # mm
d = 535  # mm
fc = 21  # MPa
fyt = 275  # MPa
lamb = 'Normal Weight Concrete'
ds = 10  # mm (diameter of stirrup)
n = 2  # number of legs
Vu = 193110  # N Ultimate Shear (Demand)

# Create an instance of the ShearDesign class
shear_design = ShearDesign(bw, d, fc, fyt, lamb, ds, n, Vu)

# Calculate the required stirrup spacing
spacing = shear_design.stirrups_spacing()

# Print the results
print(f"Concrete Shear Capacity (Vc): {shear_design.Vc:.2f} N")
print(f"Shear Carried by Stirrups (Vs): {shear_design.Vs:.2f} N")
print(f"Stirrup Spacing: {spacing}")


Concrete Shear Capacity (Vc): 143014.55 N
Shear Carried by Stirrups (Vs): 114465.45 N
Stirrup Spacing: 200
